source: https://ai.google.dev/gemma/docs/lora_tuning

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/config.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/tokenizer.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/metadata.json
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/model.weights.h5
/kaggle/input/gemma/keras/gemma_1.1_instruct_2b_en/3/assets/tokenizer/vocabulary.spm
/kaggle/input/chess-puzzle-dataset/Chess Puzzle Dataset.csv


In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [3]:
import keras
import keras_nlp

2024-07-28 14:30:35.150471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 14:30:35.150607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 14:30:35.289194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

Explore the chess dataset

In [5]:
# turn csv to dataframe
df = pd.read_csv("/kaggle/input/chess-puzzle-dataset/Chess Puzzle Dataset.csv")
df

,FEN,Player's Turn,Best Move,Explanation
0,rnbqk2r/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RN...,White,e4,This is the opening principle of controlling t...
1,r1bqkbnr/pppp1ppp/2n5/4P3/5N2/P6/1PPPPPPP/RNBQ...,Black,c6,Developing a knight is a good follow-up to e4....
2,rnbqk2r/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RN...,White,Nf3,Another developing move. This knight controls ...
3,r1bqkbnr/pppp1ppp/2n5/3Pp3/4N3/P6/1PPPPPPP/RNB...,Black,dxe4,This recaptures the pawn lost earlier. Black c...
4,r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/R...,Black,dxe4,"This captures the center pawn, challenging Whi..."
5,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,White,Nc3,White defends the e4 pawn and prepares to cast...
6,r1bqkbnr/pppp1ppp/2n5/4P3/4N2/P6/1PPPPPPP/RNBQ...,Black,Nf6,Black develops the knight and defends the d5-p...
7,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,White,d3,White prepares for castling and fights for cen...
8,r1bqkbnr/pppp2pp/2n5/4P3/3NP3/P6/1PPPPPPP/RNBQ...,Black,Bd7,Black develops the bishop and prepares ...0-0....
9,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,Black,O-O,Black castles queenside for safety and activat...


In [6]:
# modify df
df['Response'] = "The best move is " + df['Best Move'] + " because " + df['Explanation']
df

,FEN,Player's Turn,Best Move,Explanation,Response
0,rnbqk2r/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RN...,White,e4,This is the opening principle of controlling t...,The best move is e4 because This is the openin...
1,r1bqkbnr/pppp1ppp/2n5/4P3/5N2/P6/1PPPPPPP/RNBQ...,Black,c6,Developing a knight is a good follow-up to e4....,The best move is c6 because Developing a knigh...
2,rnbqk2r/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RN...,White,Nf3,Another developing move. This knight controls ...,The best move is Nf3 because Another developin...
3,r1bqkbnr/pppp1ppp/2n5/3Pp3/4N3/P6/1PPPPPPP/RNB...,Black,dxe4,This recaptures the pawn lost earlier. Black c...,The best move is dxe4 because This recaptures ...
4,r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/R...,Black,dxe4,"This captures the center pawn, challenging Whi...",The best move is dxe4 because This captures th...
5,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,White,Nc3,White defends the e4 pawn and prepares to cast...,The best move is Nc3 because White defends the...
6,r1bqkbnr/pppp1ppp/2n5/4P3/4N2/P6/1PPPPPPP/RNBQ...,Black,Nf6,Black develops the knight and defends the d5-p...,The best move is Nf6 because Black develops th...
7,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,White,d3,White prepares for castling and fights for cen...,The best move is d3 because White prepares for...
8,r1bqkbnr/pppp2pp/2n5/4P3/3NP3/P6/1PPPPPPP/RNBQ...,Black,Bd7,Black develops the bishop and prepares ...0-0....,The best move is Bd7 because Black develops th...
9,rnbqk2r/pppp1ppp/4n3/2B1p3/4P3/5N2/PPPP1PPP/RN...,Black,O-O,Black castles queenside for safety and activat...,The best move is O-O because Black castles que...


In [7]:
data = []
for index, row in df.iterrows():
    template = "Explain what is the best move as {player_turn} given the FEN position: {fen}\nResponse: {response}"
    formatted_string = template.format(
        player_turn=row["Player's Turn"],
        fen=row["FEN"],
        response=row["Response"]
    )
    data.append(formatted_string)

print(data[-1])


Explain what is the best move as White given the FEN position: 7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1
Response: The best move is hxg7+ because Double check The white pawn captures Black's queen, which was defending the square g8.


In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle 

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

Testing model without finetuning

In [9]:
prompt = template.format(
        player_turn="White",
        fen="7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1",
        response=""
    )
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

I0000 00:00:1722177142.144272      25 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1722177142.204896      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722177142.476224      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Explain what is the best move as White given the FEN position: 7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1
Response: 1. Kf6?

The position is not the best, but it is not the worst either.

I would play 1. Ke7, 2. f4, 3. Kc8+ (I think), 4. Kd7 (I think), 5. Kc8, and then the game will go 5. ... f5 6. Ke7+ and then 6. ... Kf8 (or 6. ... g5 7. f4) and the game will go 6. ... Kf8 7. Kf6+ Kf8 8. Ke6# 0-1 (I think).


In [10]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [11]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

W0000 00:00:1722177217.009658      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


34/34 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 0.4348 - sparse_categorical_accuracy: 0.4996


In [12]:
prompt = template.format(
        player_turn="White",
        fen="7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1",
        response=""
    )
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1722177283.659091      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722177283.958179      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Explain what is the best move as White given the FEN position: 7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1
Response: 1.e4 e5 2.Nf3 Nf6 3.Bc4 Nc6 4.d4 Nxe4 5.Bxe4 Qh4 6.Qxe4 Nxf3 7.Qxf3 Qe8 8.Qxe8+ Bxe8 9.Nxe6+ Kxe6 10.Nxf3 Bxf3 11.Bxf7 Nf6 12.Bxf6 Qh4 13.Qh4 Qe8 14.Nxe6 Bxe6 15.Ne4 Qe5 16.Ne6+ Kd8 17.Ne7+ Kc7 18.Qxe5 Qf7 19.Qf4 Qg7 20.Rxe5 Bxe5 21


In [13]:
prompt = template.format(
        player_turn="White",
        fen="7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1",
        response=""
    )
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1722177312.954201      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722177313.275852      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Explain what is the best move as White given the FEN position: 7k/1p1n2q1/3pNb1P/p2Ppr2/P1P5/1P2P3/6P1/4K2R w K - 0 1
Response: 1.e4 e5 2.Nf3 Nf6 3.Bc4 Nc6 4.d4 Nxe4 5.Bxe4 Qh4 6.Qxe4 Nxf3 7.Qxf3 Qe8 8.Qxe8+ Bxe8 9.Nxe6+ Kxe6 10.Nxf3 Bxf3 11.Bxf7 Nf6 12.Bxf6 Qh4 13.Qh4 Qe8 14.Nxe6 Bxe6 15.Ne4 Qe5 16.Ne6+ Kd8 17.Ne7+ Kc7 18.Qxe5 Qf7 19.Qf4 Qg7 20.Rxe5 Bxe5 21


In [14]:
prompt = template.format(
        player_turn="Black",
        fen="4r3/ppp3bk/6pp/3P1q2/P5n1/1Q3N1P/1P3PP1/R1B3K1 b - - 0 1",
        response=""
    )
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1722177343.280882      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722177343.616055      25 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Explain what is the best move as Black given the FEN position: 4r3/ppp3bk/6pp/3P1q2/P5n1/1Q3N1P/1P3PP1/R1B3K1 b - - 0 1
Response: 4...R3/p2p3k/1P4q1/6pp/3P1q2/P5n1/1Q3N1P/1P3PP1 b - - 0 1
The best move as white is 4...R3/p2p3k/1P4q1/6pp/3P1q2/P5n1/1Q3N1P/1P3PP1 b - - 0 1
The best move as white is 4...R3/p2p3k/1P4q1/6pp/1P5N/P5P/Q1N3P/1P3PP1 b - - 0 1
The best move as white is 4...R3/p2p3k/1P4q1/6


In [15]:
import pickle

In [16]:
# Save the Modle to file in the current working directory

Pkl_Filename = "gemma_lm_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(gemma_lm, file)

In [17]:
# # Load the Model back from file
# with open(Pkl_Filename, 'rb') as file:  
#     Pickled_LR_Model = pickle.load(file)

# Pickled_LR_Model